## Chapter 5. 오차역전파법(back propagation)
* 수치미분 : 단순하고 구현하기 쉽지만 계산이 오래 걸림 $\to$ 오차역전파법 : 효율적으로 계산 
    * "CS231n" 수업을 참고
* 오차역전파법(역전파) : backward propagation of errors (back propagation)

#### 5.1 계산 그래프
* 계산 그래프(computational graph) : 계산과정을 그래프로 나타낸 것
    * 노드(node)
    * 에지(edge) : 노드 사이의 직선 

* 계산 그래프를 이용한 문제풀이 흐름
    1. 계산 그래프를 구성
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행 (순전파)
        * 역전파는 이후에 미분을 계산할 때 중요한 역할을 함
    
* 계산 그래프의 특징 - 국소적 계산 : 자신과 직접 관계된 작은 범위를 계산
    * 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정도만으로 결과를 출력할 수 있음
    * 노드에서 국소적 계산을 한후 다음 노드로 전달 

* 왜 계산 그래프로 푸는가?
    * 계산 그래프의 이점
        1. 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화 할 수 있음 
        2. 중간 계산 결과를 모두 보관할 수 있음
        3. **역전파를 통해 미분을 효율적으로 계산할 수 있다**
            
#### 5.2 연쇄 법칙
* 계산 그래프의 역전파
    * 순전파 : 계산 결과를 왼쪽에서 오른쪽으로 전달
    * 역전파 : '국소적인 미분'을 순방향과는 반대인 오른쪽에서 왼쪽으로 전달
        * '국소적인 미분'의 전달원리 : 연쇄법칙(chain rule)
            * 반대방향으로 '국소적 미분'을 곱한다
    
* 연쇄법칙(chain rule)이란?
    * 합성 함수 : 여러 함수로 구성된 함수
        * ex. $z = (x+y)^2 \to z=t^2, t=x+y$
        * $\frac{\partial z}{\partial t} = 2t ,\frac{\partial t}{\partial x} = 1$
        * $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t \centerdot 1 = 2(x+y)$
        * **합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 구할 수 있음** 

* 계산그래프의 역전파 : 오른쪽에서 왼쪽으로 국소적 미분(편미분)을 곱한후 다음 노드로 전달 

#### 5.3 역전파
* 덧셈 노드의 역전파 : $z=x+y$
    * $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 1 \centerdot 1 = 1 $
    * 역방향 신호 그대로 1을 곱해서 다음 노드로 보냄
        * 순방향 입력신호가 불필요
* 곱셈 노드의 역전파 : $z=xy$
    * $\frac{\partial z}{\partial x} = y, \frac{\partial z}{\partial y} = x $
    * 상류의 값에 순전파 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보냄
        ex. x 입력값 10, y입력값이 5이고, 역전파 상류에서 1.3의 값이 흘러나온다면 x방향으로 1.3 x 5, y방향으로 1.3 x 10의 값을 흘려보냄
        * 덧셈 노드와 달리 순방향 입력신호 값이 필요
        
* 사과 쇼핑의 예 : 변수 : 사과의 가격, 사과의 개수, 소비세
    * 사과 가격에 대한 지불 금액의 미분, 사과 개수에 대한 지불 금액의 미분, 소비세에 대한 지불금액의 미분을 구해야 함
    * 빈칸에 값 채우기 풀어보기 

#### 5.4 단순한 계층 구현하기
* 신경망을 구성하는 계층 각각을 클래스로 구현
    * 계층 : 신경망의 기능 단위
        * Sigmoid,Affine등의 기능을 계층 단위로 구현
    * 곱셈 계층(MulLayer) : 초기화 필요
    * 덧셈 계층(AddLayer) : 초기화가 굳이 필요 없음
    
#### 5.5 활성화 함수 계층 구현하기
* 계산 그래프를 신경망에 적용
    * 활성화 함수 구현
        * ReLU 계층
            * $ y = x(x>0) $
            * $ y=0(x <= 0)$
            * $\frac{\partial y}{\partial x}=1(x>0), 0(x<=0) $
                * 역전파 : x가 0보다 클때는 미분값을 전달, x가 0이거나 작을때는 0을 전달 해야함
        * Sigmoid 계층
            * $y = \frac{1}{1+exp(-x)}$를 순차적으로 미분
                1. '/' 노드 (1+exp(-x)를 분모로): $y=\frac{1}{x}$을 미분 : $\frac{\partial y}{\partial x} = - \frac{1}{x^2} = -y^2$ 
                2. '+' 노드 (1 ,exp(-x) -> 1+exp(-x)) : 여과 없이 하류로 내보냄 
                3. 'exp' 노드 (-x -> exp(-x)) : $y=exp(x)$을 미분 : $\frac{\partial y}{\partial x} = exp(x)$ 
                4. 'X' 노드 (-1,x -> -x)  : 순전파 때의 값을 바꿔 곱함 
            * Sigmoid 노드로 통합 : $\frac{\partial L}{\partial y}y^2exp(-x) = \frac{\partial L}{\partial y}y(1-y)$
                * 중간 과정을 생략, 순전파의 출력만으로 역전파를 계산 가능
                
        
#### 5.6 Affine/Softmax 계층 구현하기
* Affine 계층
    * 신경망의 순전파 : 가중치 신호의 총합을 계산 (Y = np.dot(X,W)+B)
    * 어파인 변환 (Affine transformer) : 기하학에서 행렬의 곱을 뜻함 
    * 노드사이에 스칼라 값이 아닌 행렬 값이 흐르는 그래프 
    * 'dot', '+'노드 존재 
        * 행렬의 형상에 주의 : 차원의 원소수를 일치시켜야 곱할 수 있음  
            
* 배치용 Affine 계층
    1. $\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \centerdot W^T$
    2. $\frac{\partial L}{\partial W} = X^T \centerdot \frac{\partial L}{\partial Y}$
    3. $\frac{\partial L}{\partial B} = \frac{\partial L}{\partial Y}$의 첫 번째 축(0축,열방향)의 합

* Softmax-with-Loss 계층
    * Softmax : 출력층에서 사용
        * 입력값을 정규화하여(0~1사이의 확률값으로) 출력
        * 손글씨예시 : 10클래스 분류(0~10)이면 입력,출력값도 10개
    * Softmax-with-Loss : softmax함수와 손실함수인 교차엔트로피 오차를 포함하여 계층으로 표현
        * 역전파 : $(y_1-t_1,y_2-t_2,y_3-t_3)$ 정답레이블의 차분(출력과 정답레이블의 오차를 그대로 표현)
            * $(y_1,y_2,y_3)$ : Softmax 계층의 출력
            * $(t_1,t_2,t_3)$ : 정답레이블   
            
#### 5.7 오차역전파법 구현하기
* 앞 절에서 구현한 계층을 조합, 신경망을 구축

* 신경망 학습의 전체 그림
    * 전제 : 신경망에 적응 가능한 가중치와 편향이 존재, 이것들을 훈련 데이터에 학습시킴(알맞게 조정)
    * 신경망 학습 4단계
        1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져옴. 미니배치의 손실함수 값을 줄이는 것이 목표 
        2. 기울기 산출 : 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구하고, 손실함수를 최소화 하는 값을 찾음
            * 오차 역전파법을 기울기 산출에 사용 
        3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 조금씩 갱신
        4. 반복 : 1~3을 반복 
    
* 오차역전파법을 적용한 신경망 구현

* 오차역전파법으로 구한 기울기 검증
    * 수치 미분과 오차역전파법의 결과 오차가 0이 되는 일은 드뭄 : 컴퓨터의 정밀도가 유한하기 때문 
        * 올바르게 구현할 경우 0에 가까운 작은 값이 나오는 것이 정상
        
* 오차역전파법을 사용한 학습 구현 


#### 5.8 정리
* 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있음
* 계산 그래프의 노드는 국소적 계산으로 구성. 국소적 계산을 조합해 전체 계산을 구성
* 계산 그래프의 순전파는 통상의 계산을 수행. 계산 그래프의 역전파로는 각 노드의 미분을 구함
* 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있음(오차역전파법)
* 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인가능(기울기 확인)

## code

In [15]:
import numpy as np

In [1]:
# 곱셈 계층 : x,y 바꿔서 구하기
class MulLayer :
    def __init__(self) : # x,y를 초기화 : 순전파 시의 입력값을 유지하기 위해 사용 ? 
        self.x = None
        self.y = None
    
    def forward(self, x, y) :
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout) : # dout: 상류에서 넘어온 미분
        dx = dout * self.y # x와 y를 바꿔서 곱해줌
        dy = dout * self.x
        
        return dx, dy

In [2]:
# 사과 예시
apple = 100
apple_num = 2
tax = 1.1

# 계층들 
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 각 변수에 대한 미분
# backward()가 받는 인수 : 순전파의 출력에 대한 미분 

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [6]:
# 덧셈 계층 
class AddLayer:
    def __init__(self) : 
        pass
    
    def forward(self, x, y) :
        out = x + y
        return out
    
    def backward(self, dout) :
        dx = dout * 1
        dy = dout * 1
        return dx,dy
    

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층 
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파 
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax) # 이 결과로 뭘 할 수 있단거? 

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [10]:
# ReLU
# mask 인스턴스 변수 : 0,1로 구성 
class Relu :
    def __init__(self) : 
        self.mask = None 
    
    def forward(self, x) :
        self.mask = (x <= 0)   # x가 0보다 작으면 True, 크면 False 
        out = x.copy()
        out[self.mask] = 0     # x가 0 보다 작은 값은 0으로 반환 
    
    def backward(self, dout) : # 0보다 작은 값일때(True)일때는 미분값이 0을 전달해야함 
        dout[self.mask] = 0    # 0보다 작은 값의 미분값을 0으로 반환 
        dx = dout
        
        return dx
    

In [18]:
x = np.array([[1.0, -0.5],[-2.0, 3.0]])
print(x)
mask = (x<=0)
print(mask)
out = x.copy()
out[mask]

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


array([-0.5, -2. ])

In [19]:
# sigmoid
# 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용 

class Sigmoid :
    def __init__(self) :
        self.out = None
        
    def foward(self,x) :
        out = 1 / (1+ np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout) :
        dx = dout * (1.0 - self.out) * self.out
        return dx

    

In [20]:
# 배치용 Affine 예시
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W+B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [23]:
## 배치용 Affine 예시
dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [24]:
# Affine 계층 구현
class Affine:
    def __init__(self, W ,b) :
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x) :
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout) :
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
    

In [25]:
# SoftmaxWithLoss !!
class SoftmaxWithLoss :
    def __init__(self) :
        self.loss = None # 손실
        self.y = None # Softmax 출력
        self.t = None # 정답레이블 (one-hot vector)
        
    def forward(self, x, t) :
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1) :
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx     
        

In [35]:
# 오차역전파법을 적용한 신경망 구현 
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import * 
from common.gradient import numerical_gradient
from collections import OrderedDict # 순서가 있는 딕셔너리(순서를 기억)


class TwoLayerNet :
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01) :
        # 가중치 초기화 
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    
    def predict(self, x) :
        for layer in self.layers.values() :
            x = layer.forward(x)
        
        return x
    
    
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t) :
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    
    def accuracy(self, x, t) :
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    
    # x : 입력 데이터, t : 정답 레이블 
    def numerical_gradient(self, x, t) :
        loss_W = lambda W : self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

    
    def gradient(self, x, t) :
        # 순전파
        self.loss(x,t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers :
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads
    
    

In [36]:
# 오차역전파법으로 구한 기울기 검증
import sys, os
sys.path.append(os.pardir)
import numpy as np
from data.mnist import load_mnist


# 데이터 읽기
(x_train, t_train),(x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch,t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 가중치의 차이의 절대값을 구한 후, 그 절댓값들의 평균을 냄
for key in grad_numerical.keys() :
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))
    

W1:4.297928451198766e-10
b1:2.3482079938375382e-09
W2:5.962502743780398e-09
b2:1.404064551360773e-07


In [37]:
# 오차 역전파법을 사용한 학습 구현하기
import sys, os
sys.path.append(os.pardir)
import numpy as np

from data.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num) :
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차 역전파법으로 기울기를 구한다
    grad = network.gradient(x_batch,t_batch)
    
    # 갱신 
    for key in ('W1','b1','W2','b2') :
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
    

0.14366666666666666 0.1496
0.9030333333333334 0.9076
0.9207833333333333 0.9216
0.9341 0.9328
0.94245 0.9386
0.9497333333333333 0.9437
0.9558166666666666 0.9505
0.9594166666666667 0.9572
0.9640833333333333 0.9603
0.9673833333333334 0.9609
0.9710833333333333 0.9646
0.9731666666666666 0.9659
0.97475 0.967
0.9764833333333334 0.968
0.9774833333333334 0.9683
0.9775833333333334 0.9684
0.9812166666666666 0.9698
